# Write-up and code for Feb 20 and 22

## To Do:
- Write code for the interface for RL algorithms with value function approximation. The core of this interface should be a function from a (state, action) pair to a sampling of the (next state, reward) pair. It is important that this interface doesn't present the state-transition probability model or the reward model.
- Implement any Monte-Carlo Prediction algorithm with Value Function approximation
- Implement 1-step TD Prediction algorithm with Value Function approximation
- Implement Eligibility-Traces-based TD(lambda) Prediction algorithm with Value Function approximation
- Implement SARSA and SARSA(Lambda) with Value Function approximation
- Implement Q-Learning with Value Function approximation
- Optional: Implement LSTD and LSPI
- Test the above algorithms versus DP Policy Evaluation and DP Policy Iteration/Value Iteration algorithm on an example MDP
- Project Suggestion: Customize the LSPI algorithm for American Option Pricing (see this paper)